# Application Development
I will build a graphical interface that can use the model I trained in a visual format for use. This way, someone can visually see how various features affect the model's class prediction.

## Import the model

In [9]:
from sklearn.ensemble import StackingClassifier
import pickle

In [10]:
clf = pickle.load(open('../models/stacking_classifier_optimized.pkl', 'rb'))

## Import the data and mappings
I need to reference what features to include in our widget, as well as allow ourselves to map user-readable values to a prediction input.

In [15]:
import numpy as np
import pandas as pd
import json

In [19]:
# Load the test set
X_test = pd.read_csv('../data/X_test.csv')
y_test = pd.read_csv('../data/y_test.csv')
raw_map = json.load(open('../data/mappings.json', 'rb'))

In [59]:
X_test.sample(5)

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
8,28,1,791,1,1,4,2,4,1,44,...,3,3,0,5,2,2,4,2,0,2
144,22,0,1294,1,8,1,2,3,0,79,...,3,3,0,1,6,3,1,0,0,0
50,32,1,689,0,9,2,2,4,1,35,...,3,4,3,9,2,4,8,7,0,7
226,59,0,326,0,3,3,0,3,0,48,...,3,4,0,13,2,3,6,1,0,5
25,38,0,1380,1,9,2,0,3,0,75,...,3,3,0,2,3,3,2,2,2,1


In [20]:
raw_map

{'BusinessTravel': {'0': 'Travel_Rarely',
  '1': 'Travel_Frequently',
  '2': 'Non-Travel',
  '3': None,
  '4': None,
  '5': None,
  '6': None,
  '7': None,
  '8': None},
 'Department': {'0': 'Sales',
  '1': 'Research & Development',
  '2': 'Human Resources',
  '3': None,
  '4': None,
  '5': None,
  '6': None,
  '7': None,
  '8': None},
 'EducationField': {'0': 'Life Sciences',
  '1': 'Other',
  '2': 'Medical',
  '3': 'Marketing',
  '4': 'Technical Degree',
  '5': 'Human Resources',
  '6': None,
  '7': None,
  '8': None},
 'Gender': {'0': 'Female',
  '1': 'Male',
  '2': None,
  '3': None,
  '4': None,
  '5': None,
  '6': None,
  '7': None,
  '8': None},
 'JobRole': {'0': 'Sales Executive',
  '1': 'Research Scientist',
  '2': 'Laboratory Technician',
  '3': 'Manufacturing Director',
  '4': 'Healthcare Representative',
  '5': 'Manager',
  '6': 'Sales Representative',
  '7': 'Research Director',
  '8': 'Human Resources'},
 'MaritalStatus': {'0': 'Single',
  '1': 'Married',
  '2': 'Divorced

In [29]:
map = {}
for feature in raw_map.keys():
    map.update({feature: {value: int(key) for key, value in raw_map[feature].items() if value is not None}}) # Reverse key and value for easy retrieval

In [30]:
map

{'BusinessTravel': {'Travel_Rarely': 0,
  'Travel_Frequently': 1,
  'Non-Travel': 2},
 'Department': {'Sales': 0, 'Research & Development': 1, 'Human Resources': 2},
 'EducationField': {'Life Sciences': 0,
  'Other': 1,
  'Medical': 2,
  'Marketing': 3,
  'Technical Degree': 4,
  'Human Resources': 5},
 'Gender': {'Female': 0, 'Male': 1},
 'JobRole': {'Sales Executive': 0,
  'Research Scientist': 1,
  'Laboratory Technician': 2,
  'Manufacturing Director': 3,
  'Healthcare Representative': 4,
  'Manager': 5,
  'Sales Representative': 6,
  'Research Director': 7,
  'Human Resources': 8},
 'MaritalStatus': {'Single': 0, 'Married': 1, 'Divorced': 2}}

I probably should have exported the mappings in this fashion, but in order to keep the integrity of the training/test split for reproducibility from source, I will just reformat the dict when needed.

## Trying the model with a manual input

In [45]:
for column in X_test:
    if column in map.keys():
        print('mapped', '\t', column)
    else:
        print(X_test[column].dtype, '\t', column)

int64 	 Age
mapped 	 BusinessTravel
int64 	 DailyRate
mapped 	 Department
int64 	 DistanceFromHome
int64 	 Education
mapped 	 EducationField
int64 	 EnvironmentSatisfaction
mapped 	 Gender
int64 	 HourlyRate
int64 	 JobInvolvement
int64 	 JobLevel
mapped 	 JobRole
int64 	 JobSatisfaction
mapped 	 MaritalStatus
int64 	 MonthlyIncome
int64 	 MonthlyRate
int64 	 NumCompaniesWorked
bool 	 OverTime
int64 	 PercentSalaryHike
int64 	 PerformanceRating
int64 	 RelationshipSatisfaction
int64 	 StockOptionLevel
int64 	 TotalWorkingYears
int64 	 TrainingTimesLastYear
int64 	 WorkLifeBalance
int64 	 YearsAtCompany
int64 	 YearsInCurrentRole
int64 	 YearsSinceLastPromotion
int64 	 YearsWithCurrManager


In [133]:
u_input = pd.Series({
    'Age': 22,                              # int64
    'BusinessTravel': 'Non-Travel',         # mapped
    'DailyRate': 100,                       # int64
    'Department': 'Research & Development', # mapped
    'DistanceFromHome': 18,                 # int64
    'Education': 2,                         # int64
    'EducationField': 'Technical Degree',   # mapped
    'EnvironmentSatisfaction': 3,           # int64
    'Gender': 'Male',                       # mapped
    'HourlyRate': 25,                       # int64
    'JobInvolvement': 4,                    # int64
    'JobLevel': 1,                          # int64
    'JobRole': 'Laboratory Technician',     # mapped
    'JobSatisfaction': 3,                   # int64
    'MaritalStatus': 'Single',              # mapped
    'MonthlyIncome': 5238,                  # int64
    'MonthlyRate': 14812,                   # int64
    'NumCompaniesWorked': 3,                # int64
    'OverTime': False,                      # bool
    'PercentSalaryHike': 11,                # int64
    'PerformanceRating': 4,                 # int64
    'RelationshipSatisfaction': 3,          # int64
    'StockOptionLevel': 0,                  # int64
    'TotalWorkingYears': 6,                 # int64
    'TrainingTimesLastYear': 2,             # int64
    'WorkLifeBalance': 3,                   # int64
    'YearsAtCompany': 2,                    # int64
    'YearsInCurrentRole': 2,                # int64
    'YearsSinceLastPromotion': 2,           # int64
    'YearsWithCurrManager': 2               # int64
})

In [50]:
X_test['Education'].min(), X_test['Education'].max()

(1, 5)

In [51]:
X_test['EnvironmentSatisfaction'].min(), X_test['EnvironmentSatisfaction'].max()

(1, 4)

In [52]:
X_test['JobInvolvement'].min(), X_test['JobInvolvement'].max()

(1, 4)

In [53]:
X_test['JobLevel'].min(), X_test['JobLevel'].max()

(1, 5)

In [54]:
X_test['JobSatisfaction'].min(), X_test['JobSatisfaction'].max()

(1, 4)

In [72]:
X_test['MonthlyIncome'].min(), X_test['MonthlyIncome'].max(), '%0.0f' % X_test['MonthlyIncome'].median()

(1052, 19701, '5238')

In [71]:
X_test['MonthlyRate'].min(), X_test['MonthlyRate'].max(), '%0.0f' % X_test['MonthlyRate'].median()

(2094, 26959, '14812')

In [73]:
X_test['NumCompaniesWorked'].min(), X_test['NumCompaniesWorked'].max()

(1, 10)

In [74]:
X_test['PercentSalaryHike'].min(), X_test['PercentSalaryHike'].max()

(11, 25)

In [75]:
X_test['PerformanceRating'].min(), X_test['PerformanceRating'].max()

(3, 4)

In [76]:
X_test['RelationshipSatisfaction'].min(), X_test['RelationshipSatisfaction'].max()

(1, 4)

In [77]:
X_test['StockOptionLevel'].min(), X_test['StockOptionLevel'].max()

(0, 3)

In [80]:
X_test['TrainingTimesLastYear'].min(), X_test['TrainingTimesLastYear'].max()

(0, 6)

In [81]:
X_test['WorkLifeBalance'].min(), X_test['WorkLifeBalance'].max()

(1, 4)

In [82]:
X_test['YearsSinceLastPromotion'].min(), X_test['YearsSinceLastPromotion'].max()

(0, 15)

In [134]:
u_input

Age                                             22
BusinessTravel                          Non-Travel
DailyRate                                      100
Department                  Research & Development
DistanceFromHome                                18
Education                                        2
EducationField                    Technical Degree
EnvironmentSatisfaction                          3
Gender                                        Male
HourlyRate                                      25
JobInvolvement                                   4
JobLevel                                         1
JobRole                      Laboratory Technician
JobSatisfaction                                  3
MaritalStatus                               Single
MonthlyIncome                                 5238
MonthlyRate                                  14812
NumCompaniesWorked                               3
OverTime                                     False
PercentSalaryHike              

In [135]:
def convert_mappings(input_series):
    out = pd.Series()
    for feature in input_series.keys():
        if feature in map:
            out[feature] = map[feature][input_series[feature]]
        else:
            out[feature] = input_series[feature]
    return out

In [136]:
input = pd.DataFrame(convert_mappings(u_input)).transpose()
input

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,22,2,100,1,18,2,4,3,1,25,...,4,3,0,6,2,3,2,2,2,2


In [137]:
y_prob = clf.predict_proba(input)
print(f'Proba of False: {y_prob[0, 0]:.2f}% \nProba of True: {y_prob[0, 1]:.2f}% \nPrediction: {clf.predict(input).squeeze()}')

Proba of False: 0.87% 
Proba of True: 0.13% 
Prediction: False


Great. It works! Time to make widget inputs for the user to use.

P.S. This sample input is roughly based on myself.

## UI Creation

In [235]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

In [158]:
# Make a simple slider widget display to reference when using model
slider = widgets.IntSlider(description='Value:', min=0, max=100)
bar = widgets.HTML(value='', layout=widgets.Layout(width='0.3%', height='60px', background='red'))
bar_label = widgets.Label(value='0% likely to leave the company')
bar.value = '<div style="background:red; height: 60px; width: 100%"></div>'

def update_bar(change):
    value = change['new']
    bar.layout.width = f'{value * 0.8 + 0.3}%'
    bar_label.value = f'{value}% likely to leave the company'

slider.observe(update_bar, names='value')

display(widgets.VBox([slider, widgets.HBox([bar, bar_label])]))

In [195]:
# Recreate the above but with a floatProgress bar instead.
# Also, clean it up to scale it.
bar = widgets.FloatProgress(value=0.0, 
                            min=0.0, 
                            max=100.0, 
                            description='Probability:', 
                            bar_style='danger',
                            style={'bar_color': 'red'})
bar_label = widgets.Label(value='0% likely to leave the company')

all_inputs = []
slider = widgets.IntSlider(description='Value:', min=0, max=100)
all_inputs.append(slider)

def update_bar():
    value = slider.get_interact_value()
    bar.value = value
    bar_label.value = f'{value}% likely to leave the company'

slider.on_trait_change(update_bar)

display(*all_inputs, widgets.HBox([bar, bar_label]))

IntSlider(value=0, description='Value:')

In [206]:
X = pd.read_csv('../data/ibm_hr_analytics_cleaned.csv')
X.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,True,0,1102,0,1,2,0,2,0,...,3,1,0,8,0,1,6,4,0,5
1,49,False,1,279,1,8,1,0,3,1,...,4,4,1,10,3,3,10,7,1,7
2,37,True,0,1373,1,2,2,1,4,1,...,3,2,0,7,3,3,0,0,0,0
3,33,False,1,1392,1,3,4,0,4,0,...,3,3,0,8,3,3,8,7,3,0
4,27,False,0,591,1,2,1,2,1,1,...,3,4,1,6,3,3,2,2,2,2


In [290]:
# Apply it to the model with prediction probability
bar = widgets.FloatProgress(value=0.0, 
                            min=0.0, 
                            max=100.0, 
                            description='Probability:', 
                            bar_style='danger',
                            style={'bar_color': 'red',
                                   'description_width': '100px',
                                   'font_size': '20px'},
                            layout=Layout(width='70%', height='50px', margin='15px 10px 5px 0'))
bar_label = widgets.Label(value='0% likely to leave the company', layout=Layout(margin='15px 0 0 0'))

all_inputs = []

# There is certainly a more object-oriented approach for creating all these inputs, 
# but I think this approach is best for this specific project. If I were to approach
# this again on a larger project, I would calculate/store the default values once 
# then use a loop with a function call to create these.
age_slider = widgets.IntSlider(
    description='Age', 
    value=round(X['Age'].median()), 
    min=X['Age'].min(), 
    max=X['Age'].max()
)
all_inputs.append(age_slider)

business_travel_drop = widgets.Dropdown(
    description='BusinessTravel',
    options=map['BusinessTravel']
)
all_inputs.append(business_travel_drop)

dailyrate_slider = widgets.IntSlider(
    description='DailyRate',
    value=round(X['DailyRate'].median()),
    min=X['DailyRate'].min(),
    max=X['DailyRate'].max()
)
all_inputs.append(dailyrate_slider)

dept_drop = widgets.Dropdown(
    description='Department',
    options=map['Department']
)
all_inputs.append(dept_drop)

dist_slider = widgets.IntSlider(
    description='DistanceFromHome',
    value=round(X['DistanceFromHome'].median()),
    min=X['DistanceFromHome'].min(),
    max=X['DistanceFromHome'].max()
)
all_inputs.append(dist_slider)

edu_slider = widgets.IntSlider(
    description='Education',
    value=round(X['Education'].median()),
    min=X['Education'].min(),
    max=X['Education'].max()
)
all_inputs.append(edu_slider)

edu_drop = widgets.Dropdown(
    description='EducationField',
    options=map['EducationField']
)
all_inputs.append(edu_drop)

env_slider = widgets.IntSlider(
    description='EnvironmentSatisfaction',
    value=round(X['EnvironmentSatisfaction'].median()),
    min=X['EnvironmentSatisfaction'].min(),
    max=X['EnvironmentSatisfaction'].max()
)
all_inputs.append(env_slider)

gender_drop = widgets.Dropdown(
    description='Gender',
    options=map['Gender']
)
all_inputs.append(gender_drop)

hourly_rate_slider = widgets.IntSlider(
    description='HourlyRate',
    value=round(X['HourlyRate'].median()),
    min=X['HourlyRate'].min(),
    max=X['HourlyRate'].max()
)
all_inputs.append(hourly_rate_slider)

job_involvement_slider = widgets.IntSlider(
    description='JobInvolvement',
    value=round(X['JobInvolvement'].median()),
    min=X['JobInvolvement'].min(),
    max=X['JobInvolvement'].max()
)
all_inputs.append(job_involvement_slider)

job_level_slider = widgets.IntSlider(
    description='JobLevel',
    value=round(X['JobLevel'].median()),
    min=X['JobLevel'].min(),
    max=X['JobLevel'].max()
)
all_inputs.append(job_level_slider)

job_role_drop = widgets.Dropdown(
    description='JobRole',
    options=map['JobRole']
)
all_inputs.append(job_role_drop)

job_satisfaction_slider = widgets.IntSlider(
    description='JobSatisfaction',
    value=round(X['JobSatisfaction'].median()),
    min=X['JobSatisfaction'].min(),
    max=X['JobSatisfaction'].max()
)
all_inputs.append(job_satisfaction_slider)

marital_status_drop =  widgets.Dropdown(
    description='MaritalStatus',
    options=map['MaritalStatus']
)
all_inputs.append(marital_status_drop)

monthly_income_slider = widgets.IntSlider(
    description='MonthlyIncome',
    value=round(X['MonthlyIncome'].median()),
    min=X['MonthlyIncome'].min(),
    max=X['MonthlyIncome'].max()
)
all_inputs.append(monthly_income_slider)

monthly_rate_slider = widgets.IntSlider(
    description='MonthlyRate',
    value=round(X['MonthlyRate'].median()),
    min=X['MonthlyRate'].min(),
    max=X['MonthlyRate'].max()
)
all_inputs.append(monthly_rate_slider)

num_companies_worked_slider = widgets.IntSlider(
    description='NumCompaniesWorked',
    value=round(X['NumCompaniesWorked'].median()),
    min=X['NumCompaniesWorked'].min(),
    max=X['NumCompaniesWorked'].max()
)
all_inputs.append(num_companies_worked_slider)

overtime_button = widgets.Checkbox(
    description='Overtime',
    value=False
)
all_inputs.append(overtime_button)

percent_salary_hike_slider = widgets.IntSlider(
    description='PercentSalaryHike',
    value=round(X['PercentSalaryHike'].median()),
    min=X['PercentSalaryHike'].min(),
    max=X['PercentSalaryHike'].max()
)
all_inputs.append(percent_salary_hike_slider)

performance_rating_slider = widgets.IntSlider(
    description='PerformanceRating',
    value=round(X['PerformanceRating'].median()),
    min=X['PerformanceRating'].min(),
    max=X['PerformanceRating'].max()
)
all_inputs.append(performance_rating_slider)

relationship_satisfaction_slider = widgets.IntSlider(
    description='RelationshipSatisfaction',
    value=round(X['RelationshipSatisfaction'].median()),
    min=X['RelationshipSatisfaction'].min(),
    max=X['RelationshipSatisfaction'].max()
)
all_inputs.append(relationship_satisfaction_slider)

stock_option_level_slider = widgets.IntSlider(
    description='StockOptionLevel',
    value=round(X['StockOptionLevel'].median()),
    min=X['StockOptionLevel'].min(),
    max=X['StockOptionLevel'].max()
)
all_inputs.append(stock_option_level_slider)

total_working_years_slider = widgets.IntSlider(
    description='TotalWorkingYears',
    value=round(X['TotalWorkingYears'].median()),
    min=X['TotalWorkingYears'].min(),
    max=X['TotalWorkingYears'].max()
)
all_inputs.append(total_working_years_slider)

training_times_last_year_slider = widgets.IntSlider(
    description='TrainingTimesLastYear',
    value=round(X['TrainingTimesLastYear'].median()),
    min=X['TrainingTimesLastYear'].min(),
    max=X['TrainingTimesLastYear'].max()
)
all_inputs.append(training_times_last_year_slider)

work_life_balance_slider = widgets.IntSlider(
    description='WorkLifeBalance',
    value=round(X['WorkLifeBalance'].median()),
    min=X['WorkLifeBalance'].min(),
    max=X['WorkLifeBalance'].max()
)
all_inputs.append(work_life_balance_slider)

years_at_company_slider = widgets.IntSlider(
    description='YearsAtCompany',
    value=round(X['YearsAtCompany'].median()),
    min=X['YearsAtCompany'].min(),
    max=X['YearsAtCompany'].max()
)
all_inputs.append(years_at_company_slider)

years_in_current_role_slider = widgets.IntSlider(
    description='YearsInCurrentRole',
    value=round(X['YearsInCurrentRole'].median()),
    min=X['YearsInCurrentRole'].min(),
    max=X['YearsInCurrentRole'].max()
)
all_inputs.append(years_in_current_role_slider)

years_since_last_promotion_slider = widgets.IntSlider(
    description='YearsSinceLastPromotion',
    value=round(X['YearsSinceLastPromotion'].median()),
    min=X['YearsSinceLastPromotion'].min(),
    max=X['YearsSinceLastPromotion'].max()
)
all_inputs.append(years_since_last_promotion_slider)

years_with_curr_manager_slider = widgets.IntSlider(
    description='YearsWithCurrManager',
    value=round(X['YearsWithCurrManager'].median()),
    min=X['YearsWithCurrManager'].min(),
    max=X['YearsWithCurrManager'].max()
)
all_inputs.append(years_with_curr_manager_slider)

u_input = pd.Series({
    'Age': None,
    'BusinessTravel': None,
    'DailyRate': None,
    'Department': None,
    'DistanceFromHome': None,
    'Education': None,
    'EducationField': None,
    'EnvironmentSatisfaction': None,
    'Gender': None,
    'HourlyRate': None,
    'JobInvolvement': None,
    'JobLevel': None,
    'JobRole': None,
    'JobSatisfaction': None,
    'MaritalStatus': None,
    'MonthlyIncome': None,
    'MonthlyRate': None,
    'NumCompaniesWorked': None,
    'OverTime': None,
    'PercentSalaryHike': None,
    'PerformanceRating': None,
    'RelationshipSatisfaction': None,
    'StockOptionLevel': None,
    'TotalWorkingYears': None,
    'TrainingTimesLastYear': None,
    'WorkLifeBalance': None,
    'YearsAtCompany': None,
    'YearsInCurrentRole': None,
    'YearsSinceLastPromotion': None,
    'YearsWithCurrManager': None
})
def update_bar():
    for i in range(len(u_input)):
        u_input.iloc[i] = all_inputs[i].get_interact_value()
    input = pd.DataFrame(u_input).transpose()
    value = clf.predict_proba(input)[0, 1] * 100
    bar.value = value
    bar_label.value = f'{value:.2f}% likely to leave the company'

# Make interactive and display
for elem in all_inputs:
    elem.on_trait_change(update_bar)
    elem.style = {'description_width': '140px'}
    elem.layout = Layout(width='50%')
update_bar()
display(*all_inputs, widgets.HBox([bar, bar_label]))

IntSlider(value=36, description='Age', layout=Layout(width='50%'), max=60, min=18, style=SliderStyle(descripti…

Dropdown(description='BusinessTravel', layout=Layout(width='50%'), options={'Travel_Rarely': 0, 'Travel_Freque…

IntSlider(value=802, description='DailyRate', layout=Layout(width='50%'), max=1499, min=102, style=SliderStyle…

Dropdown(description='Department', layout=Layout(width='50%'), options={'Sales': 0, 'Research & Development': …

IntSlider(value=7, description='DistanceFromHome', layout=Layout(width='50%'), max=29, min=1, style=SliderStyl…

...

IntSlider(value=3, description='WorkLifeBalance', layout=Layout(width='50%'), max=4, min=1, style=SliderStyle(…

IntSlider(value=5, description='YearsAtCompany', layout=Layout(width='50%'), max=40, style=SliderStyle(descrip…

IntSlider(value=3, description='YearsInCurrentRole', layout=Layout(width='50%'), max=18, style=SliderStyle(des…

IntSlider(value=1, description='YearsSinceLastPromotion', layout=Layout(width='50%'), max=15, style=SliderStyl…

IntSlider(value=3, description='YearsWithCurrManager', layout=Layout(width='50%'), max=17, style=SliderStyle(d…